In [1]:
!pip install python-telegram-bot==13.7 --quiet
!pip install torch torchvision --quiet

In [2]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image, ImageOps
from io import BytesIO
from telegram.ext import Updater, MessageHandler, Filters, CommandHandler
from telegram import Update, InputFile
from telegram.ext.callbackcontext import CallbackContext


In [ ]:

# Token do bot do Telegram
TOKEN = 'YOUR-TOKEN-HERE'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:

# Caminho para o modelo salvo
model_path = '/content/drive/MyDrive/TrabalhoIA/Modelos/efficientnet_b2.pth'

In [7]:
# Carrega o modelo EfficientNet_B2 treinado
model = models.efficientnet_b2(pretrained=False)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
model.load_state_dict(torch.load(model_path, map_location=device))
model = model.to(device)
model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-7-04e0b8d0fa52>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions 

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [8]:

# Classes
classes = ['A', 'B']


In [9]:
# Transformações
chatbot_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Converte para 3 canais (RGB)
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalização para 3 canais (RGB)
])

In [29]:
# Função para carregar e preparar a imagem recebida pelo chatbot
def preprocess_image_for_chatbot(image):
    image = ImageOps.exif_transpose(image)
    return chatbot_transform(image).unsqueeze(0)

# Função para desfazer a normalização e converter o tensor em uma imagem PIL para exibição
def tensor_to_pil(tensor):
    tensor = tensor.squeeze(0)
    tensor = tensor * 0.5 + 0.5
    tensor = tensor.clamp(0, 1)
    pil_image = transforms.ToPILImage()(tensor)
    return pil_image

# Função de comando /start
def start(update: Update, context: CallbackContext):
    update.message.reply_text('Olá! Me envie uma imagem de uma letra A ou de uma letra B para eu dizer qual é.')

# Função para tratar imagens enviadas
def handle_image(update: Update, context: CallbackContext):
    try:
        # Obtém o arquivo da foto
        photo_file = update.message.photo[-1].get_file()

        # Baixa a foto e abre com PIL
        image = Image.open(BytesIO(photo_file.download_as_bytearray())).convert('RGB')

        # Preprocessa a imagem para o modelo
        input_tensor = preprocess_image_for_chatbot(image).to(device)

        # Converte o tensor transformado de volta para uma imagem PIL para enviar ao usuário
        transformed_image = tensor_to_pil(input_tensor.cpu())

        # Envia a imagem transformada ao usuário
        # bio = BytesIO()
        # bio.name = 'transformed_image.jpg'
        # transformed_image.save(bio, 'JPEG')
        # bio.seek(0)
        # update.message.reply_photo(photo=bio, caption="Imagem processada!")

        # Faz a previsão
        with torch.no_grad():
            output = model(input_tensor)
            _, predicted = torch.max(output, 1)
            class_name = classes[predicted.item()]

        # Responde ao usuário com a previsão
        update.message.reply_text(f'Esta imagem possui a letra "{class_name}".')
    except Exception as e:
        update.message.reply_text('Desculpe, ocorreu um erro ao processar sua imagem.')
        print(f'Erro: {e}')

# Função para tratar mensagens que não são imagens
def handle_non_image(update: Update, context: CallbackContext):
    update.message.reply_text("Olá! Me envie uma imagem de uma letra A ou de uma letra B para eu dizer qual é.")


# Função principal para configurar o bot
def main():
    updater = Updater(TOKEN, use_context=True)
    dp = updater.dispatcher

    # Comando /start
    dp.add_handler(CommandHandler('start', start))

    # Manipulador para imagens
    dp.add_handler(MessageHandler(Filters.photo, handle_image))

    # Manipulador para mensagens que não sejam imagem
    dp.add_handler(MessageHandler(~Filters.photo, handle_non_image))

    # Inicia o bot
    updater.start_polling()
    print("Bot iniciado...")
    updater.idle()

In [30]:
main()


Bot iniciado...
